In [ ]:
#🔵
main_path = '/Users/stanislavkrupnov/Jup.Notebook'

In [ ]:
#🟠
main_path = '/content/drive/Othercomputers/Mac/Jup.Notebook'

# Цель ноутбука

Выбираю лучшую бейзлайн-модель для второго этапа рек.системы - ранжирования  
первичных кандидатов внутри каждой сессии на основе признаков и таргетов.  
Выбор будет происходить между **XGBoost Ranker, CatBoost Ranker, LGBM Ranker**

# Функции

In [ ]:
def metric_eval(predictions: pd.DataFrame, valid: pd.DataFrame,
                id2type: dict) -> tuple:
    """
    Вычисляет метрики для оценки качества модели.

    Parameters:
    - predictions (pd.DataFrame): DataFrame с предсказаниями модели.
    - valid (pd.DataFrame): DataFrame с данными для валидации.
    - id2type (dict): Словарь для преобразования индексов типов в соответствующие строки.

    Returns:
    - tuple: Кортеж, содержащий локальную метрику и реколл для каждого типа.
    """
    # Преобразование индексов типов в соответствующие строки
    valid.type = valid.type.map(lambda idx: id2type[idx])

    # Создание списка с метками для каждой сессии и типа
    ground_truth = valid.groupby(['session', 'type'])['aid'].apply(list)
    ground_truth = ground_truth.reset_index().rename(columns={'aid': 'labels'})

    # Обрезка меток для типа 'clicks' до одной метки
    ground_truth.loc[ground_truth.type == 'clicks',
                     'labels'] = ground_truth.loc[ground_truth.type ==
                                                  'clicks', 'labels'].str[:1]

    # Объединение предсказаний с истинными метками
    submission_with_gt = predictions.merge(
        ground_truth[['session', 'type', 'labels']],
        how='left',
        on=['session', 'type'])

    # Отбрасывание сессий без истинных меток
    submission_with_gt = submission_with_gt[~submission_with_gt.labels_y.isna(
    )]

    # Вычисление количества совпадений между предсказанными и истинными метками
    submission_with_gt['hits'] = submission_with_gt.apply(
        lambda df: len(set(df.labels_x).intersection(set(df.labels_y))),
        axis=1)

    # Вычисление количества истинных меток для каждого типа
    submission_with_gt['gt_count'] = submission_with_gt.labels_y.str.len(
    ).clip(0, 20)

    # Вычисление реколла для каждого типа
    recall_per_type = submission_with_gt.groupby([
        'type'
    ])['hits'].sum() / submission_with_gt.groupby(['type'])['gt_count'].sum()

    # Вычисление локальной метрики на основе реколла для каждого типа
    local_validation_score = (recall_per_type * pd.Series({
        'clicks': 0.10,
        'carts': 0.30,
        'orders': 0.60
    })).sum()

    return local_validation_score, recall_per_type

In [ ]:
def read_parquets(path):
    """
    Чтение данных из файлов Parquet и их объединение в единый DataFrame.

    Parameters:
    - path (str): Путь к файлам Parquet.

    Returns:
    - tr_candidates (pd.DataFrame): Объединенный DataFrame.
    """
    # Список файлов Parquet
    file_list = glob.glob(path)

    # Создание пустого DataFrame для сбора данных
    tr_candidates = pd.DataFrame()

    # Цикл для чтения и объединения файлов
    for file in tqdm(file_list):
        # Чтение файла Parquet во временный DataFrame
        df_ = pd.read_parquet(file)

        # Объединение временного DataFrame с основным датасетом
        tr_candidates = pd.concat([tr_candidates, df_], ignore_index=True)

    return tr_candidates

# Import

In [ ]:
import os
import sys
from tqdm import tqdm
import glob
import pyarrow.parquet as pq
import pickle
import pandas as pd
import gc
import numpy as np
import itertools
from datetime import datetime as dt
import polars as pl
import shutil
import xgboost as xgb
from sklearn.model_selection import GroupKFold
from imblearn.under_sampling import RandomUnderSampler
import joblib

In [ ]:
import lightgbm as lgb
from lightgbm.sklearn import LGBMRanker

In [ ]:
import gdown
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


# XGBoost Baseline

## Тренировка

In [ ]:
ver_folder = 'xgb_base'
random_state = 42
# slice_value = 1
downsamp_k = 16

In [ ]:
# Перебор типов 'clicks', 'carts', 'orders'
for typee in tqdm(['clicks', 'carts', 'orders'], desc='type'):

    # Путь к файлам Parquet для текущего типа
    path = f'{main_path}/tr/ver_6/tr_candidates_{typee}/*'

    # Чтение данных из файлов Parquet
    tr_candidates = read_parquets(path)
    print('Чтение данных завершено')

    # Преобразование типов данных
    tr_candidates.session = tr_candidates.session.astype(np.int32)
    tr_candidates.aid = tr_candidates.aid.astype(np.int32)

    # Расчет соотношения классов
    class_ratio = tr_candidates['target'].value_counts(
    )[1] / tr_candidates['target'].value_counts()[0]

    # Указание желаемого соотношения классов
    under_sampler = RandomUnderSampler(sampling_strategy=class_ratio *
                                       downsamp_k)

    # Undersampling классов для лучшего баланса
    FEATURES = tr_candidates.columns[:-1]
    X = tr_candidates.loc[:, FEATURES]
    y = tr_candidates.loc[:, 'target']
    X_train, y_train = under_sampler.fit_resample(X, y)
    tr_candidates = pd.merge(X_train,
                             y_train,
                             left_index=True,
                             right_index=True)
    del X, y
    print('Undersampling классов завершено')

    # Обучение модели XGBoost
    folder_path = f'model/baseline/{ver_folder}'
    os.makedirs(folder_path, exist_ok=True)

    # Сортировка и сброс индексов
    tr_candidates = tr_candidates.sort_values('session')
    tr_candidates = tr_candidates.reset_index(drop=True)

    FEATURES = tr_candidates.columns[:-1]

    X_train = tr_candidates[FEATURES]
    y_train = tr_candidates['target']

    # Группировка по сессиям и подсчет числа действий
    group_sizes_tr = X_train.groupby('session')[['aid']].count()

    # Удаление ненужных столбцов
    X_train = X_train.iloc[:, 2:]

    # Подготовка данных для XGBoost
    dtrain = xgb.DMatrix(X_train, y_train, group=group_sizes_tr.aid.tolist())

    # Параметры XGBoost
    xgb_params = {'objective': 'rank:pairwise', 'tree_method': 'gpu_hist'}

    # Обучение модели
    model = xgb.train(xgb_params, dtrain=dtrain, verbose_eval=100)

    # Сохранение модели
    model.save_model(f'model/baseline/{ver_folder}/XGB_base_{typee}.xgb')
    print('training_done')

    # Сохранение модели на диск
    # folder_path = f'{main_path}/models/{ver_folder}/'
    # os.makedirs(folder_path, exist_ok=True)

    # file_list = glob.glob(f'/content/model/*{typee}.xgb')
    # [shutil.copy(file, f'{main_path}/models/{ver_folder}/') for file in file_list]

100%|██████████| 5/5 [00:21<00:00,  4.25s/it]


reading_done
sampling_done


type:  33%|███▎      | 1/3 [01:16<02:33, 76.56s/it]

training_done



100%|██████████| 5/5 [00:44<00:00,  8.93s/it]


...

training_done



100%|██████████| 5/5 [00:46<00:00,  9.20s/it]


reading_done
sampling_done


type: 100%|██████████| 3/3 [04:43<00:00, 94.57s/it]

training_done


## Предсказание + оценка

In [ ]:
lv_predictions_full = pd.DataFrame()

# Перебор типов 'clicks', 'carts', 'orders'
for typee in tqdm(['clicks', 'carts', 'orders'], desc='type'):

    # Проверка наличия директории с кандидатами
    if not os.path.exists(f'/content/test_candidates_{typee}'):
        # Импорт кандидатов выбранного типа
        shutil.copytree(f'{main_path}/lv/ver_6/test_candidates_{typee}/',
                        f'/content/test_candidates_{typee}/')

    # Путь к файлам Parquet для текущего типа
    path = f'test_candidates_{typee}/*'

    # Чтение данных из файлов Parquet
    test_candidates = read_parquets(path)
    test_candidates = test_candidates.sort_values('session')
    test_candidates = test_candidates.reset_index(drop=True)

    FEATURES = test_candidates.columns[2:]

    # Подготовка массива для предсказаний
    preds = np.zeros(len(test_candidates))

    # Загрузка обученной модели
    model = xgb.Booster()
    model.load_model(f'model/baseline/{ver_folder}/XGB_base_{typee}.xgb')
    model.set_param({'predictor': 'gpu_predictor'})

    # Подготовка данных для XGBoost
    dtest = xgb.DMatrix(data=test_candidates[FEATURES])

    # Предсказание
    preds = model.predict(dtest)

    # Формирование DataFrame с предсказаниями
    lv_predictions = test_candidates[['session', 'aid']].copy()
    lv_predictions['pred'] = preds
    lv_predictions = lv_predictions.sort_values(
        ['session', 'pred'], ascending=[True, False]).reset_index(drop=True)
    lv_predictions['n'] = lv_predictions.groupby(
        'session').aid.cumcount().astype('int8')
    lv_predictions = lv_predictions.loc[lv_predictions.n < 20]
    lv_predictions = lv_predictions.groupby('session').aid.apply(list)
    lv_predictions = lv_predictions.to_frame().reset_index()
    lv_predictions.rename(columns={'aid': 'labels'}, inplace=True)
    lv_predictions['type'] = typee

    # Объединение временного DataFrame с основным датасетом
    lv_predictions_full = pd.concat([lv_predictions_full, lv_predictions],
                                    ignore_index=True)
    del lv_predictions, preds, dtest, test_candidates

# lv_predictions.to_parquet(f'lv_predictions_{typee}.parquet')
# shutil.copy2(f'lv_predictions_{typee}.parquet', f'{main_path}/lv/{ver_folder}/lv_predictions_{typee}.parquet' )

type: 100%|██████████| 3/3 [02:38<00:00, 52.97s/it]


In [ ]:
id2type_name = 'id2type.pkl'
# version = 'ver_6.1'

In [ ]:
valid = pd.read_parquet(f'{main_path}/s/test_labels_loc.parquet')

with open(f'{main_path}/pkl/{id2type_name}', 'rb') as file:
    id2type = pickle.load(file)

print('Model score :', metric_eval(lv_predictions_full, valid, id2type))

Model score : (0.5220552152693079, type
carts     0.365164
clicks    0.386253
orders    0.623134
dtype: float64)


In [ ]:
best_score_base = {'xgb': 0.5220552152693079}

# LGBM Baseline

## Тренировка

In [ ]:
ver_folder = 'lgbm_base'
random_state = 42
# slice_value = 1
downsamp_k = 16

In [ ]:
for typee in tqdm(['clicks', 'carts', 'orders'], desc='type'):

    # Загрузка предварительно сэмплированных кандидатов
    tr_candidates = pd.read_parquet(
        f'{main_path}/tr/ver_6/downsampled/tr_candidates_{typee}.parquet')
    print('reading_done')

    # В данном участке кода отключены downsample и перевод в int32,
    # поскольку они не используются
    # tr_candidates.session = tr_candidates.session.astype(np.int32)
    # tr_candidates.aid = tr_candidates.aid.astype(np.int32)

    # Downsampling трен.датасета
    # class_ratio = tr_candidates['target'].value_counts()[1] / \
    #               tr_candidates['target'].value_counts()[0]
    # under_sampler = RandomUnderSampler(sampling_strategy=class_ratio * downsamp_k)
    # FEATURES = tr_candidates.columns[:-1]
    # X = tr_candidates.loc[:, FEATURES]
    # y = tr_candidates.loc[:, 'target']
    # X_train, y_train = under_sampler.fit_resample(X, y)
    # tr_candidates = pd.merge(X_train, y_train, left_index=True, right_index=True)
    # del X, y
    print('sampling_done')

    # Обучение модели LGBMRanker
    folder_path = f'model/baseline/{ver_folder}'
    os.makedirs(folder_path, exist_ok=True)

    tr_candidates = tr_candidates.sort_values('session')
    tr_candidates = tr_candidates.reset_index(drop=True)

    FEATURES = tr_candidates.columns[:-1]

    # Создание объекта LGBMRanker
    ranker = LGBMRanker(objective="lambdarank")

    X_train = tr_candidates[FEATURES]
    y_train = tr_candidates['target']

    group_id_tr = X_train.groupby('session').size().tolist()
    X_train = X_train.iloc[:, 2:]  # Убираю ['session','aid'] для обучения

    # Обучение ранкера
    ranker = ranker.fit(X_train, tr_candidates['target'], group=group_id_tr)

    # Сохранение модели
    folder_path = f'model/baseline/{ver_folder}'
    os.makedirs(folder_path, exist_ok=True)
    joblib.dump(ranker, f'model/baseline/{ver_folder}/LGB_base_{typee}.joblib')
    print('training_done')

# Сохранение модели на диск
# folder_path = f'{main_path}/models/{ver_folder}/'
# os.makedirs(folder_path, exist_ok=True)

# file_list = glob.glob(f'/content/model/*{typee}.xgb')
# [shutil.copy(file, f'{main_path}/models/{ver_folder}/') for file in file_list]

## Предсказание + оценка

In [ ]:
lv_predictions_full = pd.DataFrame()

for typee in tqdm(['clicks', 'carts', 'orders'], desc='type'):

    if os.path.exists(f'/content/test_candidates_{typee}') == False:
        # импорт кандидатов выбранного типа
        shutil.copytree(f'{main_path}/lv/ver_6/test_candidates_{typee}/',
                        f'/content/test_candidates_{typee}/')

    path = f'test_candidates_{typee}/*'
    test_candidates = read_parquets(path)
    test_candidates = test_candidates.sort_values('session')
    test_candidates = test_candidates.reset_index(drop=True)

    # Создание предсказаний
    FEATURES = test_candidates.columns[2:]
    model = joblib.load(f'model/baseline/{ver_folder}/LGB_base_{typee}.joblib')
    preds = model.predict(test_candidates[FEATURES])

    # Получение топ-20 предсказаний для каждой сессии и доп преобразования
    lv_predictions = test_candidates[['session', 'aid']].copy()
    lv_predictions['pred'] = preds
    lv_predictions = lv_predictions.sort_values(
        ['session', 'pred'], ascending=[True, False]).reset_index(drop=True)
    lv_predictions['n'] = lv_predictions.groupby(
        'session').aid.cumcount().astype('int8')
    lv_predictions = lv_predictions.loc[lv_predictions.n < 20]
    lv_predictions = lv_predictions.groupby('session').aid.apply(list)
    lv_predictions = lv_predictions.to_frame().reset_index()
    lv_predictions.rename(columns={'aid': 'labels'}, inplace=True)
    lv_predictions['type'] = typee

    # Объединение временного DataFrame с основным датасетом
    lv_predictions_full = pd.concat([lv_predictions_full, lv_predictions],
                                    ignore_index=True)
    del lv_predictions, preds, test_candidates

    # lv_predictions.to_parquet(f'lv_predictions_{typee}.parquet')
    # shutil.copy2(f'lv_predictions_{typee}.parquet', f'{main_path}/lv/{ver_folder}/lv_predictions_{typee}.parquet' )

type: 100%|██████████| 3/3 [05:40<00:00, 113.36s/it]


In [ ]:
id2type_name = 'id2type.pkl'
# version = 'ver_6.1'

In [ ]:
valid = pd.read_parquet(f'{main_path}/s/test_labels_loc.parquet')

with open(f'{main_path}/pkl/{id2type_name}', 'rb') as file:
    id2type = pickle.load(file)

print('Model score :', metric_eval(lv_predictions_full, valid, id2type))

Model score : (0.5248555778378172, type
carts     0.364534
clicks    0.391417
orders    0.627256
dtype: float64)


In [ ]:
best_score_base['lgbm'] = 0.5248555778378172
best_score_base

{'xgb': 0.5220552152693079, 'lgbm': 0.5248555778378172}

# Catboost Baseline

## Тренировка

In [ ]:
ver_folder = 'catb_base'
random_state = 42
# slice_value = 1
downsamp_k = 16

In [ ]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 17.5 MB/s eta 0:00:00


In [ ]:
import catboost
from catboost import CatBoostRanker, Pool

In [ ]:
import os
os.environ['CATBOOST_GPU'] = '1'

In [ ]:
# Итерация по типам событий ('clicks', 'carts', 'orders')
for typee in tqdm(['clicks', 'carts', 'orders'], desc='type'):

    # Путь к данным тренировочных кандидатов выбранного типа
    path = f'{main_path}/tr/ver_6/tr_candidates_{typee}/*'

    # Чтение данных тренировочных кандидатов из Parquet-файлов
    tr_candidates = read_parquets(path)
    print('reading_done')

    # Приведение типов колонок 'session' и 'aid' к int32
    tr_candidates.session = tr_candidates.session.astype(np.int32)
    tr_candidates.aid = tr_candidates.aid.astype(np.int32)

    # Расчет соотношения классов
    class_ratio = tr_candidates['target'].value_counts(
    )[1] / tr_candidates['target'].value_counts()[0]

    # Указание желаемого соотношения классов для RandomUnderSampler
    under_sampler = RandomUnderSampler(sampling_strategy=class_ratio *
                                       downsamp_k)

    # Отбор признаков
    FEATURES = tr_candidates.columns[:-1]

    # Разделение на признаки и целевую переменную
    X = tr_candidates.loc[:, FEATURES]
    y = tr_candidates.loc[:, 'target']

    # Применение RandomUnderSampler для балансировки классов
    X_train, y_train = under_sampler.fit_resample(X, y)
    tr_candidates = pd.merge(X_train,
                             y_train,
                             left_index=True,
                             right_index=True)
    del X, y
    print('sampling_done')

    # Создание директории для сохранения моделей
    folder_path = f'model/baseline/{ver_folder}'
    os.makedirs(folder_path, exist_ok=True)

    # Сортировка и сброс индексов в тренировочных кандидатах
    tr_candidates = tr_candidates.sort_values('session')
    tr_candidates = tr_candidates.reset_index(drop=True)

    # Отбор признаков
    FEATURES = tr_candidates.columns[:-1]

    # Разделение на признаки и целевую переменную
    X_train = tr_candidates[FEATURES]
    y_train = tr_candidates['target']

    # Создание группы для CatBoostRanker
    group_id_tr = X_train.iloc[:, 0]

    # Удаление ненужной колонки 'session' для обучения
    X_train = X_train.iloc[:, 2:]

    # Создание Pool для CatBoost
    catboost_train = Pool(data=X_train, label=y_train, group_id=group_id_tr)

    # Инициализация и обучение CatBoostRanker
    ranker = CatBoostRanker(loss_function='PairLogit',
                            task_type='GPU',
                            random_state=42)
    ranker.fit(catboost_train, early_stopping_rounds=50)

    # Создание директории для сохранения моделей CatBoost
    folder_path = f'model/baseline/{ver_folder}'
    os.makedirs(folder_path, exist_ok=True)

    # Сохранение модели CatBoostRanker на диск
    ranker.save_model(f'model/baseline/{ver_folder}/catb_base_{typee}')
    print('training_done')

100%|██████████| 5/5 [00:39<00:00,  7.94s/it]


reading_done
sampling_done
0:	learn: 0.6722801	total: 52.4ms	remaining: 52.3s
1:	learn: 0.6521243	total: 105ms	remaining: 52.6s
2:	learn: 0.6345881	total: 156ms	remaining: 51.8s
3:	learn: 0.6183704	total: 207ms	remaining: 51.6s
4:	learn: 0.6027706	total: 257ms	remaining: 51.2s
5:	learn: 0.5885391	total: 304ms	remaining: 50.4s
6:	learn: 0.5755264	total: 352ms	remaining: 49.9s
7:	learn: 0.5636078	total: 400ms	remaining: 49.6s
8:	learn: 0.5530807	total: 448ms	remaining: 49.4s
9:	learn: 0.5431993	total: 496ms	remaining: 49.1s
10:	learn: 0.5343235	total: 544ms	remaining: 49s
11:	learn: 0.5259114	total: 592ms	remaining: 48.8s
12:	learn: 0.5179888	total: 639ms	remaining: 48.5s
13:	learn: 0.5109231	total: 687ms	remaining: 48.4s
14:	learn: 0.5044133	total: 734ms	remaining: 48.2s
15:	learn: 0.4982586	total: 782ms	remaining: 48.1s
16:	learn: 0.4923669	total: 830ms	remaining: 48s
17:	learn: 0.4870210	total: 878ms	remaining: 47.9s
18:	learn: 0.4823054	total: 924ms	remaining: 47.7s
19:	learn: 0.4779

type:  33%|███▎      | 1/3 [02:20<04:40, 140.46s/it]

training_done



100%|██████████| 5/5 [00:32<00:00,  6.52s/it]


...

999:	learn: 0.3026234	total: 35.1s	remaining: 0us
training_done



100%|██████████| 5/5 [00:32<00:00,  6.42s/it]


reading_done
sampling_done
0:	learn: 0.6463423	total: 36.2ms	remaining: 36.2s
1:	learn: 0.6018590	total: 71.2ms	remaining: 35.5s
2:	learn: 0.5639152	total: 106ms	remaining: 35.3s
3:	learn: 0.5284154	total: 142ms	remaining: 35.4s
4:	learn: 0.4974022	total: 177ms	remaining: 35.2s
5:	learn: 0.4694077	total: 212ms	remaining: 35.1s
6:	learn: 0.4444772	total: 246ms	remaining: 34.9s
7:	learn: 0.4217797	total: 280ms	remaining: 34.7s
8:	learn: 0.4027890	total: 314ms	remaining: 34.6s
9:	learn: 0.3850198	total: 347ms	remaining: 34.4s
10:	learn: 0.3682255	total: 380ms	remaining: 34.2s
11:	learn: 0.3541375	total: 414ms	remaining: 34.1s
12:	learn: 0.3411670	total: 447ms	remaining: 33.9s
13:	learn: 0.3297538	total: 479ms	remaining: 33.7s
14:	learn: 0.3194655	total: 512ms	remaining: 33.6s
15:	learn: 0.3102211	total: 543ms	remaining: 33.4s
16:	learn: 0.3019609	total: 576ms	remaining: 33.3s
17:	learn: 0.2943388	total: 608ms	remaining: 33.2s
18:	learn: 0.2876094	total: 640ms	remaining: 33s
19:	learn: 0.2

type: 100%|██████████| 3/3 [06:12<00:00, 124.17s/it]

training_done


## Предсказание + оценка

In [ ]:
lv_predictions_full = pd.DataFrame()

for typee in tqdm(['clicks', 'carts', 'orders'], desc='type'):

    if os.path.exists(f'/content/test_candidates_{typee}') == False:
        # импорт кандидатов выбранного типа
        shutil.copytree(f'{main_path}/lv/ver_6/test_candidates_{typee}/',
                        f'/content/test_candidates_{typee}/')

    path = f'test_candidates_{typee}/*'
    test_candidates = read_parquets(path)
    test_candidates = test_candidates.sort_values('session')
    test_candidates = test_candidates.reset_index(drop=True)

    # Создание предсказаний
    FEATURES = test_candidates.columns[2:]
    dtest = Pool(data=test_candidates[FEATURES])
    preds = np.zeros(len(test_candidates))
    model = CatBoostRanker(random_state=42)
    model.load_model(f'model/baseline/{ver_folder}/catb_base_{typee}')
    preds = model.predict(dtest)

    # Получение топ-20 предсказаний для каждой сессии и доп преобразования
    lv_predictions = test_candidates[['session', 'aid']].copy()
    lv_predictions['pred'] = preds
    lv_predictions = lv_predictions.sort_values(
        ['session', 'pred'], ascending=[True, False]).reset_index(drop=True)
    lv_predictions['n'] = lv_predictions.groupby(
        'session').aid.cumcount().astype('int8')
    lv_predictions = lv_predictions.loc[lv_predictions.n < 20]
    lv_predictions = lv_predictions.groupby('session').aid.apply(list)
    lv_predictions = lv_predictions.to_frame().reset_index()
    lv_predictions.rename(columns={'aid': 'labels'}, inplace=True)
    lv_predictions['type'] = typee

    # Объединение временного DataFrame с основным датасетом
    lv_predictions_full = pd.concat([lv_predictions_full, lv_predictions],
                                    ignore_index=True)
    del lv_predictions, preds, test_candidates

    # lv_predictions.to_parquet(f'lv_predictions_{typee}.parquet')
    # shutil.copy2(f'lv_predictions_{typee}.parquet', f'{main_path}/lv/{ver_folder}/lv_predictions_{typee}.parquet' )

type: 100%|██████████| 3/3 [03:32<00:00, 70.85s/it]


In [ ]:
id2type_name = 'id2type.pkl'
# version = 'ver_6.1'

In [ ]:
valid = pd.read_parquet(f'{main_path}/s/test_labels_loc.parquet')

with open(f'{main_path}/pkl/{id2type_name}', 'rb') as file:
    id2type = pickle.load(file)

print('Model score :', metric_eval(lv_predictions_full, valid, id2type))

Model score : (0.5271976703740328, type
carts     0.368052
clicks    0.398249
orders    0.628262
dtype: float64)


In [ ]:
best_score_base = {
    'xgb': 0.5220552152693079,
    'lgbm': 0.5248555778378172,
    'catb': 0.5271976703740328
}

## Тренировка с кросс-валидацией

In [ ]:
for typee in tqdm(['clicks', 'carts', 'orders'], desc='type'):

    path = f'{main_path}/tr/ver_6/tr_candidates_{typee}/*'
    tr_candidates = read_parquets(path)
    print('reading_done')

    tr_candidates.session = tr_candidates.session.astype(np.int32)
    tr_candidates.aid = tr_candidates.aid.astype(np.int32)

    class_ratio = tr_candidates['target'].value_counts()[1] / \
                  tr_candidates['target'].value_counts()[0]

    # Указываем желаемое соотношение классов
    under_sampler = RandomUnderSampler(sampling_strategy=class_ratio *
                                       downsamp_k)

    # downsamling для лучшего баланса классов
    FEATURES = tr_candidates.columns[:-1]
    X = tr_candidates.loc[:, FEATURES]
    y = tr_candidates.loc[:, 'target']
    X_train, y_train = under_sampler.fit_resample(X, y)
    tr_candidates = pd.merge(X_train,
                             y_train,
                             left_index=True,
                             right_index=True)
    tr_candidates = tr_candidates.sort_values('session')
    tr_candidates = tr_candidates.reset_index(drop=True)

    del X, y
    print('sampling_done')

    # Обучение модели
    skf = GroupKFold(n_splits=5)
    FEATURES = tr_candidates.columns[:-1]

    scores = []

    for fold, (train_idx, valid_idx) in enumerate(
            skf.split(tr_candidates,
                      tr_candidates['target'],
                      groups=tr_candidates['session'])):

        X_train = tr_candidates.loc[train_idx, FEATURES]
        y_train = tr_candidates.loc[train_idx, 'target']
        X_valid = tr_candidates.loc[valid_idx, FEATURES]
        y_valid = tr_candidates.loc[valid_idx, 'target']

        group_id_tr = X_train.iloc[:, 0]
        group_id_v = X_valid.iloc[:, 0]

        X_train = X_train.iloc[:, 2:]
        X_valid = X_valid.iloc[:, 2:]

        catboost_train = Pool(data=X_train,
                              label=y_train,
                              group_id=group_id_tr)
        catboost_valid = Pool(data=X_valid, label=y_valid, group_id=group_id_v)

        ranker = CatBoostRanker(loss_function='PairLogit',
                                task_type='GPU',
                                random_state=42)
        ranker.fit(catboost_train,
                   eval_set=catboost_valid,
                   early_stopping_rounds=50,
                   verbose=False)

        # Рассчитать метрику pairlogit
        pairlogit = ranker.best_score_['validation']['PairLogit']
        scores.append(pairlogit)

        folder_path = f'model/baseline/{ver_folder}_kfolds'
        os.makedirs(folder_path, exist_ok=True)
        ranker.save_model(
            f'model/baseline/{ver_folder}_kfolds/catb_base_{fold}_{typee}')
    # Усреднить pairlogit по всем фолдам
    pairlogit_mean = np.mean(scores)
    print(f'pairlogit_mean_{typee}:', pairlogit_mean)

    # Сохранение модели на диск
    # folder_path = f'{main_path}/models/{ver_folder}/'
    # os.makedirs(folder_path, exist_ok=True)

    # file_list = glob.glob(f'/content/model/*{typee}.xgb')
    # [shutil.copy(file, f'{main_path}/models/{ver_folder}/') for file in file_list]

100%|██████████| 5/5 [00:20<00:00,  4.00s/it]


reading_done
sampling_done


type:  33%|███▎      | 1/3 [05:14<10:28, 314.04s/it]

pairlogit_mean_clicks: 0.39510939463998984



100%|██████████| 5/5 [00:34<00:00,  6.94s/it]


...

pairlogit_mean_carts: 0.3072526368101608



100%|██████████| 5/5 [00:33<00:00,  6.79s/it]


reading_done
sampling_done


type: 100%|██████████| 3/3 [13:55<00:00, 278.63s/it]

pairlogit_mean_orders: 0.19181008929776722


## Pairlogit's каждого типа Cross-val

 - pairlogit_mean_clicks: 0.395109
 - pairlogit_mean_carts: 0.307252
 - pairlogit_mean_orders: 0.191810

## Предсказание + оценка (кросс-валидация

In [ ]:
lv_predictions_full = pd.DataFrame()

for typee in tqdm(['clicks', 'carts', 'orders'], desc='type'):

    if os.path.exists(f'/content/test_candidates_{typee}') == False:
        # импорт кандидатов выбранного типа
        shutil.copytree(f'{main_path}/lv/ver_6/test_candidates_{typee}/',
                        f'/content/test_candidates_{typee}/')

    path = f'test_candidates_{typee}/*'
    test_candidates = read_parquets(path)
    test_candidates = test_candidates.sort_values('session')
    test_candidates = test_candidates.reset_index(drop=True)

    FEATURES = test_candidates.columns[2:]
    dtest = Pool(data=test_candidates[FEATURES])
    preds = np.zeros(len(test_candidates))
    for fold in tqdm(range(5)):
        model = CatBoostRanker(random_state=42)
        model.load_model(
            f'model/baseline/{ver_folder}_kfolds/catb_base_{fold}_{typee}')
        fold_preds = model.predict(dtest)
        preds += fold_preds / 5

    lv_predictions = test_candidates[['session', 'aid']].copy()
    lv_predictions['pred'] = preds
    lv_predictions = lv_predictions.sort_values(
        ['session', 'pred'], ascending=[True, False]).reset_index(drop=True)
    lv_predictions['n'] = lv_predictions.groupby(
        'session').aid.cumcount().astype('int8')
    lv_predictions = lv_predictions.loc[lv_predictions.n < 20]
    lv_predictions = lv_predictions.groupby('session').aid.apply(list)
    lv_predictions = lv_predictions.to_frame().reset_index()
    lv_predictions.rename(columns={'aid': 'labels'}, inplace=True)
    lv_predictions['type'] = typee

    # Объединение временного DataFrame с основным датасетом
    lv_predictions_full = pd.concat([lv_predictions_full, lv_predictions],
                                    ignore_index=True)
    del lv_predictions, preds, test_candidates

    # lv_predictions.to_parquet(f'lv_predictions_{typee}.parquet')
    # shutil.copy2(f'lv_predictions_{typee}.parquet', f'{main_path}/lv/{ver_folder}/lv_predictions_{typee}.parquet' )

100%|██████████| 5/5 [00:07<00:00,  1.43s/it]

100%|██████████| 5/5 [00:07<00:00,  1.44s/it]

100%|██████████| 5/5 [00:07<00:00,  1.43s/it]

type: 100%|██████████| 3/3 [05:37<00:00, 112.50s/it]


In [ ]:
id2type_name = 'id2type.pkl'
# version = 'ver_6.1'

In [ ]:
valid = pd.read_parquet(f'{main_path}/s/test_labels_loc.parquet')

with open(f'{main_path}/pkl/{id2type_name}', 'rb') as file:
    id2type = pickle.load(file)

print('Model score :', metric_eval(lv_predictions_full, valid, id2type))

Model score : (0.5271169524492465, type
carts     0.368120
clicks    0.397638
orders    0.628195
dtype: float64)


# Вывод

In [ ]:
best_score_base = {
    'xgb': 0.5220552152693079,
    'lgbm': 0.5248555778378172,
    'catb': 0.5271169524492465
}

**Лучшую метрику показал CatBoost Ranker**  
Продолжу его тюнинг